In [1]:
!pwd

/JT/jose/FL_AM_Defect-Detection/notebooks


In [2]:
# import os
# os.chdir('..')

In [3]:
!python3 -m pip install --force-reinstall pandas
# !pip install wheel

  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2023.3.post1
    Uninstalling pytz-2023.3.post1:
      Successfully uninstalled pytz-2023.3.post1
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2023.3
    Uninstalling tzdata-2023.3:
      Successfully uninstalled tzdata-2023.3
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24

In [4]:
!ls

 Copy_of_LSTM_Feature_RUL_Offset.ipynb
 HARDataset
 KFOLD_LSTM_Feature_RUL_Offset.ipynb
'NEW DATA LSTM_Feature_RUL_Offset-Copy2.ipynb'
'NEW DATA. ShuffleOff LSTM_MODEL RUL-Copy1.ipynb'
'NEW DATA. ShuffleOff LSTM_MODEL RUL.ipynb'
'NEW DATA. ShuffleOff MLP_MODEL RUL.ipynb'
'NEW DATA. ShuffleOn LSTM_MODEL RUL.ipynb'
'NEW DATA. ShuffleOn MLP_MODEL RUL  .ipynb'
'NEW DATA_RF_Feature_RUL_Offset_FROMGOOGLEDRIVE-WITH .ipynb'
 NEWDATA_OFFSET_MODEL.ipynb
 PM_test.txt
 RF_Feature_RUL_Offset_FROMGOOGLEDRIVE.ipynb
'SHORT NEW DATA_1DCNN AND MLP_RF_Feature_RUL_Offset_FROMGOOGLEDRIVE-WITH .ipynb'
 _lessvarsLSTM_Feature_RUL_Offset.ipynb
 lessvarsLSTM_Feature_RUL_Offset-Copy1.ipynb
 out_server_4_fei.txt


# MODEL 1 RANDOM FOREST VS MLP!

In [5]:
#! pip3 install auto-sklearn

import os
os.chdir('..')

import pandas as pd
import os
import numpy as np
RNDSEED = np.random.seed(39)
PRECISION = 4 # 3 of digits to keep after the decimal point
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score, mean_squared_error
RUNNING_ON_COLAB = False # we assume running on CoLab! Change to False if running locally.

# *italicized text*

## mount gdrive
#from google.colab import drive#
#drive.mount("/content/gdrive")

# If running locally, define current working path
path = os.getcwd()

# If Google colab
if RUNNING_ON_COLAB:
    path = "/content/gdrive/My Drive/Colab Notebooks/RandomForest_FeatureData"

print(path)

# define current data path. This is after we did classification. We have done some
# cleaning already.
data_path =  'fl_testbed/version2/data/initial'
print(data_path)
#/home/jose/FL_AM_Defect-Detection
#/home/jose/FL_AM_Defect-Detection/fl_testbed/version2/data/initial

## Play code

# <font color=cyan>The key is to transform the time column! </font>


# !cd 'notebooks/fl_testbed/version2/data/initial' && ls

## start - play code
print(data_path)
csv_file = data_path + '/combined_offset_misalignment.csv'
df_temp = pd.read_csv(csv_file, chunksize=50000) 
df = pd.concat(df_temp, ignore_index=True)

print(df.columns)

df.head(5)

#238722
#DO NOT FORGET SORT VALUES
df1=df.loc[0:238721].sort_values(by='wf_start_time').reset_index()
df2=df.loc[0+238722:0+238722+238722-1].sort_values(by='wf_start_time').reset_index()
df3=df.loc[0+238722+238722:].sort_values(by='wf_start_time').reset_index()

lists=[]
for df in [df1,df2,df3]:
    # Let's find the youngest & oldest timestamp

    df['wf_start_time'] = pd.to_datetime(df['wf_start_time']) # make sure it is datetime

    youngest = min(df.wf_start_time)
    oldest = max(df.wf_start_time)
    print(youngest)
    print(oldest)
    span = oldest - youngest
    print(span)
    print(span.total_seconds())

    ## Using Oldest - current to determine the RUL
    df['rul'] = df['wf_start_time'].apply(lambda x: (oldest - x).total_seconds())
    lists.append(df)



/JT/jose/FL_AM_Defect-Detection
fl_testbed/version2/data/initial
fl_testbed/version2/data/initial
Index(['Unnamed: 0', 'wf_start_time', 'S1_CrestFactor_g~g', 'S1_DerivedPeak_g',
       'S1_Peak~Peak_g', 'S1_RMS_g', 'S1_TruePeak_g', 'S1_HighFrequency_grms',
       'S1_Kurtosis_g~g', 'S2_CrestFactor_g~g', 'S2_DerivedPeak_g',
       'S2_Peak~Peak_g', 'S2_RMS_g', 'S2_TruePeak_g', 'S2_HighFrequency_grms',
       'S2_Kurtosis_g~g', 'S1_temp', 'status'],
      dtype='object')
2022-08-09 14:03:22.000007
2022-09-01 20:11:50.000007
23 days 06:08:28
2009308.0
2022-08-09 14:03:22.000007
2022-09-01 20:11:50.000007
23 days 06:08:28
2009308.0
2022-08-09 14:03:22.000007
2022-09-01 20:11:50.000007
23 days 06:08:28
2009308.0


In [6]:
#THIS MEAN THE M3
df=pd.concat(lists[-1:],ignore_index=True)



# drop Unnamed columns
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]


In [7]:


df.loc[0:238721]




,index,wf_start_time,S1_CrestFactor_g~g,S1_DerivedPeak_g,S1_Peak~Peak_g,S1_RMS_g,S1_TruePeak_g,S1_HighFrequency_grms,S1_Kurtosis_g~g,S2_CrestFactor_g~g,S2_DerivedPeak_g,S2_Peak~Peak_g,S2_RMS_g,S2_TruePeak_g,S2_HighFrequency_grms,S2_Kurtosis_g~g,S1_temp,status,rul
0,591557,2022-08-09 14:03:22.000007,12.3214,0.5170,8.3688,0.3655,4.5040,0.1355,29.4402,11.3516,0.8401,12.4549,0.5941,6.7436,0.1040,37.6491,72.4676,No_Offset,2009308.0
1,591558,2022-08-09 14:03:23.000007,5.5933,2.5483,20.0644,1.8019,10.0786,1.9368,8.9174,5.6735,1.8216,13.1957,1.2881,7.3079,0.2197,5.0971,72.4427,No_Offset,2009307.0
2,591559,2022-08-09 14:03:24.000007,4.0140,2.5484,14.3660,1.8020,7.2333,1.6188,5.0306,7.7328,1.2040,11.7569,0.8513,6.5833,0.3566,12.4059,72.4770,No_Offset,2009306.0
3,591560,2022-08-09 14:03:25.000007,2.6487,0.6943,2.5795,0.4910,1.3004,0.4809,2.2368,3.7461,0.2835,1.4845,0.2004,0.7508,0.1578,2.8937,72.4549,No_Offset,2009305.0
4,591561,2022-08-09 14:03:26.000007,2.5022,0.6752,2.3616,0.4774,1.1946,0.4748,2.2111,3.7041,0.2932,1.4761,0.2073,0.7679,0.1620,2.7997,72.4988,No_Offset,2009304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238717,643079,2022-09-01 20:11:29.000007,2.5083,0.6880,2.3624,0.4865,1.2203,0.4834,2.1624,3.5082,0.2728,1.2713,0.1929,0.6767,0.1447,2.7364,73.5386,12.7mm_Offset,21.0
238718,643080,2022-09-01 20:11:30.000007,2.3925,0.6879,2.2706,0.4864,1.1638,0.4837,2.1555,3.2138,0.2856,1.2836,0.2020,0.6491,0.1457,2.7667,73.5767,12.7mm_Offset,20.0
238719,643081,2022-09-01 20:11:31.000007,5.7067,0.2556,2.0422,0.1808,1.0315,0.0308,13.5090,5.3715,0.1918,1.3841,0.1356,0.7285,0.0184,3.9787,73.5347,12.7mm_Offset,19.0
238720,643082,2022-09-01 20:11:32.000007,4.2261,0.0406,0.2402,0.0287,0.1215,0.0115,3.1910,4.3151,0.1361,0.7445,0.0963,0.4154,0.0036,3.2765,73.5728,12.7mm_Offset,18.0


In [8]:

df.loc[0+238722:0+238722+238722-1]



,index,wf_start_time,S1_CrestFactor_g~g,S1_DerivedPeak_g,S1_Peak~Peak_g,S1_RMS_g,S1_TruePeak_g,S1_HighFrequency_grms,S1_Kurtosis_g~g,S2_CrestFactor_g~g,S2_DerivedPeak_g,S2_Peak~Peak_g,S2_RMS_g,S2_TruePeak_g,S2_HighFrequency_grms,S2_Kurtosis_g~g,S1_temp,status,rul


In [9]:

df.loc[0+238722+238722:]


,index,wf_start_time,S1_CrestFactor_g~g,S1_DerivedPeak_g,S1_Peak~Peak_g,S1_RMS_g,S1_TruePeak_g,S1_HighFrequency_grms,S1_Kurtosis_g~g,S2_CrestFactor_g~g,S2_DerivedPeak_g,S2_Peak~Peak_g,S2_RMS_g,S2_TruePeak_g,S2_HighFrequency_grms,S2_Kurtosis_g~g,S1_temp,status,rul


In [10]:



df.columns



# df.head()

df.loc[0:238721]


df.loc[0+238722:0+238722+238722-1]


df.loc[0+238722+238722:] #NOW THIS IS VALIDATION DATA

## RF Regression

## Common imports
import pandas as pd
import numpy as np
import csv
import os

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score, mean_squared_error
# from pandas_ml import ConfusionMatrix

import matplotlib
import matplotlib.pyplot as plt
plt.style.available
%matplotlib inline

# to make this notebook's output stable across runs
RNDSEED = np.random.seed(39)

### Exploring the data a bit & prep the df

# ## future, let's read the file
# csv_file = data_path + '/combined_offset_misalignment_with_RUL.csv'
# df_temp = pd.read_csv(csv_file, chunksize=50000) 
# big_df = pd.concat(df_temp)

#df = big_df # reset 








#THIS IS THE BEGINNING OF THE FEDERATED CODE!!!!!

# drop unwanted cols
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] # drop Unnamed
df = df[df.columns.drop(list(df.filter(regex='wf_start_time')))] # drop time column
df = df[df.columns.drop(list(df.filter(regex='rul')))] # drop status column
df = df[df.columns.drop(list(df.filter(regex='index')))] # drop status column



df.shape

df.columns





status_map = {
    "No_Offset": 0,
    "1mm_Offset": 1,
    "3mm_Offset": 2,
    "6.25mm_Offset": 3,
    "12.7mm_Offset": 4,
}

df["status"] = df["status"].map(status_map)



In [11]:





### RF works

# Get X & y
# Naming convention: X as predictors; y as response.
from sklearn.model_selection import train_test_split

y = df['status'] # pop response
X = df.drop('status',axis = 1) # drop response



print (X.shape)
print (X.columns)

## true orignal ones
# data split


#STARTING POINT OF train_cut_split_2!!!




# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = RNDSEED)

# Use the same function above for the validation set
X_train, X_vals, y_train, y_vals = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= RNDSEED,shuffle=True) # 0.25 x 0.8 = 0.2






y_test.shape

X_train.dtypes


y_train.dtypes





(238722, 15)
Index(['S1_CrestFactor_g~g', 'S1_DerivedPeak_g', 'S1_Peak~Peak_g', 'S1_RMS_g',
       'S1_TruePeak_g', 'S1_HighFrequency_grms', 'S1_Kurtosis_g~g',
       'S2_CrestFactor_g~g', 'S2_DerivedPeak_g', 'S2_Peak~Peak_g', 'S2_RMS_g',
       'S2_TruePeak_g', 'S2_HighFrequency_grms', 'S2_Kurtosis_g~g', 'S1_temp'],
      dtype='object')


dtype('int64')

In [12]:
df.head()

,S1_CrestFactor_g~g,S1_DerivedPeak_g,S1_Peak~Peak_g,S1_RMS_g,S1_TruePeak_g,S1_HighFrequency_grms,S1_Kurtosis_g~g,S2_CrestFactor_g~g,S2_DerivedPeak_g,S2_Peak~Peak_g,S2_RMS_g,S2_TruePeak_g,S2_HighFrequency_grms,S2_Kurtosis_g~g,S1_temp,status
0,12.3214,0.5170,8.3688,0.3655,4.5040,0.1355,29.4402,11.3516,0.8401,12.4549,0.5941,6.7436,0.1040,37.6491,72.4676,0
1,5.5933,2.5483,20.0644,1.8019,10.0786,1.9368,8.9174,5.6735,1.8216,13.1957,1.2881,7.3079,0.2197,5.0971,72.4427,0
2,4.0140,2.5484,14.3660,1.8020,7.2333,1.6188,5.0306,7.7328,1.2040,11.7569,0.8513,6.5833,0.3566,12.4059,72.4770,0
3,2.6487,0.6943,2.5795,0.4910,1.3004,0.4809,2.2368,3.7461,0.2835,1.4845,0.2004,0.7508,0.1578,2.8937,72.4549,0
4,2.5022,0.6752,2.3616,0.4774,1.1946,0.4748,2.2111,3.7041,0.2932,1.4761,0.2073,0.7679,0.1620,2.7997,72.4988,0


# RANDOM FOREST PART!

In [13]:
from sklearn.metrics import mean_squared_error,roc_auc_score

n_estimators = 10 # vary small one, done within 1 min;

def rf_regression ():
    rf = RandomForestClassifier(
    n_estimators=n_estimators,      
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    max_features='auto',
    random_state = RNDSEED,verbose=2
        
  )  
    rf.fit(X_train, y_train)
  
    return rf

rf = rf_regression()


InvalidParameterError: The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'log2', 'sqrt'} or None. Got 'auto' instead.

In [ ]:


### Save/Load the model

# ## utility - save/load the model
# import os
# import joblib

# # to save
# joblib.dump(rf, data_path + '/random_forest_offset_RUL.joblib') 

# # to load
# rf = joblib.load(data_path + '/random_forest_offset_RUL.joblib')

### Evaluation

# y_pred = rf.predict(X_vals) ## using the untinted dataset!
    
y_prob = rf.predict_proba(X_vals)

macro_roc_auc_ovo = roc_auc_score(y_vals, y_prob, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    y_vals, y_prob, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(y_vals, y_prob, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    y_vals, y_prob, multi_class="ovr", average="weighted"
)
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)
## with n_estimators = 150

# R^2: 0.9991654290573937
# Mean Absolute Error (MAE): 1158.6216816072117
# Mean Squared Error (MSE): 333818221.94037396
# Root Mean Squared Error (RMSE): 18270.692979205083
# Explained Variance Score: 0.9991654316522736
# Max Error: 1998236.0866666667
# Mean Squared Log Error: 0.012717975468372306
# Median Absolute Error: 147.4266666667536

## n_estimators = 10
# R^2: 0.9990822069686238
# Mean Absolute Error (MAE): 1213.5715068447246
# Mean Squared Error (MSE): 367106044.79767376
# Mean Absolute Percentage Error (MAPE): 1.0690696275079867e+17
# Root Mean Squared Error (RMSE): 19160.011607451437
# Explained Variance Score: 0.9990822141398445
# Max Error: 1998818.5
# Mean Squared Log Error: 0.012296657672986336
# Median Absolute Error: 137.60000000009313

# y_pred = rf.predict(X_test) ## using the untinted dataset!
    
y_prob = rf.predict_proba(X_test)

macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    y_test, y_prob, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    y_test, y_prob, multi_class="ovr", average="weighted"
)
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)
## with n_estimators = 150

# R^2: 0.9991654290573937
# Mean Absolute Error (MAE): 1158.6216816072117
# Mean Squared Error (MSE): 333818221.94037396
# Root Mean Squared Error (RMSE): 18270.692979205083
# Explained Variance Score: 0.9991654316522736
# Max Error: 1998236.0866666667
# Mean Squared Log Error: 0.012717975468372306
# Median Absolute Error: 147.4266666667536

## n_estimators = 10
# R^2: 0.9990822069686238
# Mean Absolute Error (MAE): 1213.5715068447246
# Mean Squared Error (MSE): 367106044.79767376
# Mean Absolute Percentage Error (MAPE): 1.0690696275079867e+17
# Root Mean Squared Error (RMSE): 19160.011607451437
# Explained Variance Score: 0.9990822141398445
# Max Error: 1998818.5
# Mean Squared Log Error: 0.012296657672986336
# Median Absolute Error: 137.60000000009313

rf.feature_importances_

plt.barh(df.columns[:-1], rf.feature_importances_)

# Done with the RF PART!

In [14]:




from sklearn.preprocessing import StandardScaler,LabelBinarizer
lbz = LabelBinarizer()


scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
y_train=lbz.fit_transform(y_train)



scaler=StandardScaler()
X_vals=scaler.fit_transform(X_vals)
y_vals=lbz.fit_transform(y_vals)


scaler=StandardScaler()
X_test=scaler.fit_transform(X_test)
y_test=lbz.fit_transform(y_test)





In [15]:
# X_train,y_train

In [16]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [17]:

from tensorflow.keras.layers import Dropout,Conv1D,MaxPooling1D,Flatten, Activation, Dense
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.layers import Dropout,Conv1D,MaxPooling1D,Flatten, Activation
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from autosklearn.regression import AutoSklearnRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L2,L1,L1L2
import tensorflow as tf



2023-10-13 18:06:48.687079: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:



# MLP



path


# !rm -rf logs/fit/*
# !rm -rf /home/jose/FL_AM_Defect-Detection/checkpoint/RUL16/*


log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# def scheduler(epoch, lr):
#       if epoch < 40:
#         lr=0.0001
#         return lr
#       else:
#         lr=0.00001
#         return lr #* tf.math.exp(-0.1)
# lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 10**-7 * 10**(epoch/3))

es=EarlyStopping(
    monitor="val_loss",

    patience=6,
    verbose=1,
    mode="auto",
    min_delta=0.0001
)

filepath=path+'/checkpoint/OFFSET16/'+"weights-improvement-BEST_OFFSET.h5"

# fname = os.path.sep.join([filepath,args["weights"],"weights-{epoch:03d}-{val_loss:.4f}.h5"])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_accuracy',
    

    mode='auto',
    save_best_only=True, verbose=1)


# # checkpoint_filepath = "./tmp/checkpoint"

#scaler = MinMaxScaler()



%load_ext tensorboard
%reload_ext tensorboard







#TRAINING
tf.keras.backend.clear_session()



# self.model
model = Sequential()

# hidden layers
model.add(Dense(60, activation="relu", input_dim=X_train.shape[1],kernel_regularizer=L1L2(l2=0.001,l1=0.001))) #Better
#GOOD 60

model.add(Dropout(0.5))


model.add(Dense(30, activation="relu",kernel_regularizer=L2(l2=0.001)))  #Better
#GOOD 40
model.add(Dropout(0.5))

# model.add(Dense(40, activation="relu"))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation="relu",kernel_regularizer=L2(l2=0.001)))  #Better
# output layer
model.add(Dense(5, activation="softmax"))
              # softmax for probability, #values are sigmoid

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(
#         learning_rate=self.learning_rate),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

# Compile the network :
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# NN_model.summary()

# Configure the model and start training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

# y_train

#FAST AI SEE IF TRIANING IMPROVES !
# 1420492

history=model.fit(X_train, y_train,epochs=13,batch_size=32,validation_data=(X_vals,y_vals) ,verbose=1,callbacks=[tensorboard_callback,lr,model_checkpoint_callback,es],)


logs/fit/20231013-180650


2023-10-13 18:06:50.656022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 26451 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


Epoch 1/13


2023-10-13 18:06:52.104080: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f956d703bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-13 18:06:52.104129: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-10-13 18:06:52.110506: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-13 18:06:52.129322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-13 18:06:52.268415: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4475/4476 [============================>.] - ETA: 0s - loss: 2.2445 - accuracy: 0.1405
Epoch 1: val_accuracy improved from -inf to 0.13595, saving model to /JT/jose/FL_AM_Defect-Detection/checkpoint/OFFSET16/weights-improvement-BEST_OFFSET.h5
4476/4476 [==============================] - 23s 5ms/step - loss: 2.2446 - accuracy: 0.1405 - val_loss: 2.0533 - val_accuracy: 0.1360 - lr: 1.0000e-07
Epoch 2/13
  36/4476 [..............................] - ETA: 20s - loss: 2.2453 - accuracy: 0.1380

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4468/4476 [============================>.] - ETA: 0s - loss: 2.2263 - accuracy: 0.1430
Epoch 2: val_accuracy improved from 0.13595 to 0.13800, saving model to /JT/jose/FL_AM_Defect-Detection/checkpoint/OFFSET16/weights-improvement-BEST_OFFSET.h5
4476/4476 [==============================] - 21s 5ms/step - loss: 2.2261 - accuracy: 0.1431 - val_loss: 2.0275 - val_accuracy: 0.1380 - lr: 2.1544e-07
Epoch 3/13
4467/4476 [============================>.] - ETA: 0s - loss: 2.1876 - accuracy: 0.1482
Epoch 3: val_accuracy improved from 0.13800 to 0.14251, saving model to /JT/jose/FL_AM_Defect-Detection/checkpoint/OFFSET16/weights-improvement-BEST_OFFSET.h5
4476/4476 [==============================] - 21s 5ms/step - loss: 2.1876 - accuracy: 0.1482 - val_loss: 1.9746 - val_accuracy: 0.1425 - lr: 4.6416e-07
Epoch 4/13
4465/4476 [============================>.] - ETA: 0s - loss: 2.1090 - accuracy: 0.1619
Epoch 4: val_accuracy improved from 0.14251 to 0.16980, saving model to /JT/jose/FL_AM_Defect-Det

In [19]:
model.summary()

# model.save(path+'MLP_REG_RUL_16',save_format='h5')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                960       
                                                                 
 dropout (Dropout)           (None, 60)                0         
                                                                 
 dense_1 (Dense)             (None, 30)                1830      
                                                                 
 dropout_1 (Dropout)         (None, 30)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 155       
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
history.history

{'loss': [2.244567394256592,
  2.226142644882202,
  2.1875810623168945,
  2.108877182006836,
  1.9598276615142822,
  1.7329457998275757,
  1.422351598739624,
  1.0498381853103638,
  0.7242278456687927,
  0.516847550868988,
  0.39006075263023376,
  0.3011271357536316,
  0.2506581246852875],
 'accuracy': [0.14049234986305237,
  0.14308254420757294,
  0.14821408689022064,
  0.16184930503368378,
  0.20561745762825012,
  0.3297307789325714,
  0.5383643507957458,
  0.6974070072174072,
  0.8233286142349243,
  0.8952259421348572,
  0.9284377694129944,
  0.9516867995262146,
  0.9608536958694458],
 'val_loss': [2.0532970428466797,
  2.0275278091430664,
  1.9745759963989258,
  1.871934413909912,
  1.699304223060608,
  1.4413665533065796,
  1.0368894338607788,
  0.6261825561523438,
  0.4160296618938446,
  0.32159966230392456,
  0.26158803701400757,
  0.20649930834770203,
  0.18023636937141418],
 'val_accuracy': [0.13595141470432281,
  0.13800397515296936,
  0.14250706136226654,
  0.169797882437706

In [20]:



# loading the saved model
loaded_model = tf.keras.models.load_model('/home/jose/FL_AM_Defect-Detection/checkpoint/OFFSET16/weights-improvement-BEST_OFFSET.h5')





OSError: No file or directory found at /home/jose/FL_AM_Defect-Detection/checkpoint/OFFSET16/weights-improvement-BEST_OFFSET.h5

In [ ]:
import visualkeras

In [ ]:

visualkeras.layered_view(loaded_model, legend=True,  draw_volume=True,spacing=50) 

In [ ]:
from sklearn.metrics import classification_report


# y_pred = loaded_model.predict(X_test) ## using the untinted dataset!

y_prob = loaded_model.predict(X_test)
y_prob


In [ ]:
# # np.argmax(model.predict(X_test),axis=1)
# acc_score=classification_report(y_test, np.argmax(y_prob,axis=1))
# print("acc_score: ",acc_score)

In [ ]:



macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    y_test, y_prob, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    y_test, y_prob, multi_class="ovr", average="weighted"
)
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)
## with n_estimators = 150

# R^2: 0.9991654290573937
# Mean Absolute Error (MAE): 1158.6216816072117
# Mean Squared Error (MSE): 333818221.94037396
# Root Mean Squared Error (RMSE): 18270.692979205083
# Explained Variance Score: 0.9991654316522736
# Max Error: 1998236.0866666667
# Mean Squared Log Error: 0.012717975468372306
# Median Absolute Error: 147.4266666667536

## n_estimators = 10
# R^2: 0.9990822069686238
# Mean Absolute Error (MAE): 1213.5715068447246
# Mean Squared Error (MSE): 367106044.79767376
# Mean Absolute Percentage Error (MAPE): 1.0690696275079867e+17
# Root Mean Squared Error (RMSE): 19160.011607451437
# Explained Variance Score: 0.9990822141398445
# Max Error: 1998818.5
# Mean Squared Log Error: 0.012296657672986336
# Median Absolute Error: 137.60000000009313